In [151]:
import folium
import pandas as pd
import numpy as np
from sympy import *
import matplotlib.pyplot as plt

DAEGU = [35.85, 128.56]
KADIZ = [(39.00, 123.30),
         (37.00, 124.00),
         (30.00, 124.00),
         (30.00, 124.00),
         (30.00, 125.25),
         (30.00, 125.25),
         (32.30, 126.50),
         (32.30, 127.30),
         (34.17, 128.52),
         (35.13, 129.48),
         (36.00, 130.30),
         (37.17, 133.00),
         (39.00, 133.00),
         (39.00, 123.30)]
IEODO = [32.11753829417425, 125.16666665729801]
DOKDO = [37.24315823449135, 131.86690646281266]

def init_map():
    m = folium.Map(location = DAEGU, zoom_start = 5.5)
    folium.PolyLine(locations = KADIZ,tooltip = 'KADIZ', color='red').add_to(m)
    folium.Marker(IEODO, popup="<i>IEODO</i>").add_to(m)
    folium.Marker(DOKDO, popup="<i>DOKDO</i>").add_to(m)
    return m

m = init_map()

m

In [118]:
def mk_row(point):
    x, y = point
    return [y * y, x, y, x * y, 1]

#return value (a, b, c, d, e, f) matches (ax^2 + by^2 + cx + by + exy + f = 0)
def find_ellipse(points):
    if len(points) != 5:
        raise ValueError
    else:
        mat = []
        constant = []
        for point in points:
            mat.append(mk_row(point))
            constant.append(-point[0]**2)
        sol = np.linalg.solve(np.array(mat), constant)
    return np.concatenate((np.array([1]), sol), axis = None)

def make_route(sol, data, input_axis, select):
    x = symbols('x', real = True)
    y = symbols('y', real = True)
    a, b, c, d, e, f = sol
    route = []
    if input_axis == 'x':
        for i in data:
            x = i
            ans = solve(a*x**2 + b*y**2 + c*x + d*y + e*x*y + f)
            if len(ans) == 0:
                print('No real solution at {}'.format(i))
                break
            if select == 'max':
                y_sol = max(ans)
                route.append([y_sol, i])
            elif select == 'min':
                y_sol = min(ans)
                route.append([y_sol, i])
    elif input_axis == 'y':
        for i in data:
            y = i
            ans = solve(a*x**2 + b*y**2 + c*x + d*y + e*x*y + f)
            if len(ans) == 0:
                print('No real solution at {}'.format(i))
                break
            if select == 'max':
                x_sol = max(ans)
                route.append([i, x_sol])
            elif select == 'min':
                x_sol = min(ans)
                route.append([i, x_sol])
    return route

In [119]:
# Make base route
points = [(120.15, 29.331), (125.8, 31.369), (127.42, 32.459), (129.44, 34.436), (131.35, 38.448)]

sol = find_ellipse(points)

lat_range = np.linspace(123, 131, 100)
base_route = np.array(make_route(sol, lat_range, 'x', 'min'))

# print(len(route_1), route_1)

In [120]:
folium.PolyLine(locations = base_route, tooltip = 'flight tracks', color = 'blue').add_to(m)
m

In [121]:
# make noise
mu, sigma = 0.1, 0.01
route_dataset = []

for i in range(100):
    s = np.random.default_rng(i).normal(mu, sigma, size=(100, 2))
    route_dataset.append(base_route + s)

route_dataset = np.array(route_dataset)
print(route_dataset.shape)
# print(route_dataset[0])

(100, 100, 2)


In [122]:
# show one route in dataset 
folium.PolyLine(locations = route_dataset[0], tooltip = 'track with noise', color = 'orange').add_to(m)
m

In [123]:
# Split datasets
ratio = 0.7
boundary = int(len(route_dataset) * ratio)

train_dataset = route_dataset[:boundary, :, :]
test_dataset = route_dataset[boundary:, :, :]

print(train_dataset.shape, test_dataset.shape)

(70, 100, 2) (30, 100, 2)


In [131]:
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler , MinMaxScaler

class RouteDataset(Dataset): 
    def __init__(self, train=True):    
        # Scaling
        self.minmax = MinMaxScaler()
        self.standard = StandardScaler()
        self.dataset = train_dataset if train == True else test_dataset
        
        self.x_data = standard.fit_transform(self.dataset.reshape(-1, self.dataset.shape[-1])).reshape(self.dataset.shape)
        self.y_data = minmax.fit_transform(self.dataset.reshape(-1, self.dataset.shape[-1])).reshape(self.dataset.shape)
            
    def __len__(self): 
        return len(self.x_data)
    
    def __getitem__(self, idx): 
        x = torch.FloatTensor(self.x_data[idx])
        y = torch.FloatTensor(self.y_data[idx])
        return x, y

In [132]:
batch_size = 16

train_loader = DataLoader(dataset=RouteDataset(train=True), 
                          batch_size=batch_size, 
                          shuffle=True)
test_loader = DataLoader(dataset=RouteDataset(train=False), 
                         batch_size=batch_size, 
                         shuffle=True)

In [148]:
# Model definition
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

device = 'cpu'

class LSTM(nn.Module):
    def __init__(self,num_classes,input_size,hidden_size,num_layers,seq_length):
        super(LSTM,self).__init__()
        self.num_classes = num_classes
        self.num_layers = num_layers
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.seq_length = seq_length

        self.lstm = nn.LSTM(input_size=input_size , hidden_size=hidden_size , num_layers=num_layers, batch_first=True)
        self.fc_1 = nn.Linear(hidden_size,128)
        self.fc = nn.Linear(128,num_classes)

        self.relu = nn.ReLU()

    def forward(self,x):
        h_0 = Variable(torch.zeros(self.num_layers, x.size(0),self.hidden_size)).to(device)
        c_0 = Variable(torch.zeros(self.num_layers, x.size(0),self.hidden_size)).to(device)

        output, (hn,cn) = self.lstm(x,(h_0,c_0))

        hn = hn.view(-1, self.hidden_size)
        out = self.relu(hn)
        out = self.fc_1(out)
        out = self.relu(out)
        out = self.fc(out)
        return out


In [149]:
num_epochs = 1000
learning_rate = 0.001

input_size = 2
hidden_size = 4
num_layers = 1
num_classes = 2
seq_length = 100

print_every = 100
n_iter = 0

model = LSTM(num_classes, input_size, hidden_size, num_layers, seq_length)
loss_function = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [150]:
# Model training
for epoch in range(num_epochs):
    for i, (x_train, y_train) in enumerate(train_loader):
        x_train = Variable(x_train.view(-1, seq_length, input_size))
        y_train = Variable(y_train)
        
        optimizer.zero_grad()
        outputs = model.forward(x_train)
#         import pdb; pdb.set_trace()
        loss = loss_function(outputs, y_train)
        
        loss.backward()
        optimizer.step()
        n_iter += 1
        
        if epoch % print_every == 0:
            print("Epoch : %d, loss : %1.5f" % (epoch, loss.item()))

> /var/folders/c9/y3tyld_57fx2xmp_h_mhwwfr0000gn/T/ipykernel_55808/1282784493.py(10)<module>()
      8         outputs = model.forward(x_train)
      9         import pdb; pdb.set_trace()
---> 10         loss = loss_function(outputs, y_train)
     11 
     12         loss.backward()

ipdb> outputs.shape
torch.Size([16, 2])
ipdb> y_train.shape
torch.Size([16, 100, 2])
ipdb> quit


BdbQuit: 

# 아래는 옛날 코드

In [34]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import pandas as pd

device = 'cpu'

In [35]:
class LSTM(nn.Module):
    def __init__(self,num_classes,input_size,hidden_size,num_layers,seq_length):
        super(LSTM,self).__init__()
        self.num_classes = num_classes
        self.num_layers = num_layers
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.seq_length = seq_length

        self.lstm = nn.LSTM(input_size = input_size , hidden_size = hidden_size , num_layers = num_layers, batch_first = True)
        self.fc_1 = nn.Linear(hidden_size,128)
        self.fc = nn.Linear(128,num_classes)

        self.relu = nn.ReLU()

    def forward(self,x):
        h_0 = Variable(torch.zeros(self.num_layers, x.size(0),self.hidden_size)).to(device)
        c_0 = Variable(torch.zeros(self.num_layers, x.size(0),self.hidden_size)).to(device)

        output, (hn,cn) = self.lstm(x,(h_0,c_0))

        hn = hn.view(-1,self.hidden_size)
        out = self.relu(hn)
        out = self.fc_1(out)
        out = self.relu(out)
        out = self.fc(out)
        return out


In [36]:
df = pd.DataFrame(route)
y = df.iloc[:,0:]

df[0] = df[0].astype(float)

print(df.dtypes)
print(df.shape)

0    float64
1    float64
dtype: object
(100, 2)


In [37]:
def split_train_test(data,test_ratio):
    train_set_size = int(len(data)*(1 - test_ratio))
    train_indices = data[:train_set_size]
    test_indices = data[train_set_size:]
    return train_indices,test_indices

train_set,test_set = split_train_test(df.values,0.2)

In [38]:
from sklearn.preprocessing import StandardScaler , MinMaxScaler

minmax = MinMaxScaler()
standard = StandardScaler()
x_ss = standard.fit_transform(df)
y_mm = minmax.fit_transform(y)
x_train, x_test = split_train_test(x_ss,0.)
y_train, y_test = split_train_test(y_mm,0.)

x_train_tensors = Variable(torch.Tensor(x_train))
x_test_tensors = Variable(torch.Tensor(x_test))
y_train_tensors = Variable(torch.Tensor(y_train))
y_test_tensors = Variable(torch.Tensor(y_test))
x_train_final = torch.reshape(x_train_tensors,(x_train_tensors.shape[0],1,x_train_tensors.shape[1]))
x_test_final = torch.reshape(x_test_tensors,(x_test_tensors.shape[0],1,x_test_tensors.shape[1]))

# print(x_train_final.shape, x_train_final)
# print(y_train_tensors.shape, y_train_tensors)

In [39]:
num_epochs = 6000
learning_rate = 0.0001
input_size = 2
hidden_size = 4
num_layers = 1
num_classes = 2

model = LSTM(num_classes, input_size, hidden_size, num_layers, x_train_final.shape[1]).to(device)

loss_function = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [40]:
for epoch in range(num_epochs):
    outputs = model.forward(x_train_final.to(device))
    optimizer.zero_grad()
    loss = loss_function(outputs, y_train_tensors.to(device))
    
    loss.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print("Epoch : %d, loss : %1.5f" % (epoch,loss.item()))

Epoch : 0, loss : 0.35840
Epoch : 100, loss : 0.21322
Epoch : 200, loss : 0.13704
Epoch : 300, loss : 0.10042
Epoch : 400, loss : 0.08266
Epoch : 500, loss : 0.07169
Epoch : 600, loss : 0.06211
Epoch : 700, loss : 0.05170
Epoch : 800, loss : 0.03995
Epoch : 900, loss : 0.02781
Epoch : 1000, loss : 0.01782
Epoch : 1100, loss : 0.01122
Epoch : 1200, loss : 0.00750
Epoch : 1300, loss : 0.00572
Epoch : 1400, loss : 0.00473
Epoch : 1500, loss : 0.00396
Epoch : 1600, loss : 0.00331
Epoch : 1700, loss : 0.00272
Epoch : 1800, loss : 0.00225
Epoch : 1900, loss : 0.00184
Epoch : 2000, loss : 0.00150
Epoch : 2100, loss : 0.00114
Epoch : 2200, loss : 0.00086
Epoch : 2300, loss : 0.00067
Epoch : 2400, loss : 0.00053
Epoch : 2500, loss : 0.00043
Epoch : 2600, loss : 0.00035
Epoch : 2700, loss : 0.00029
Epoch : 2800, loss : 0.00024
Epoch : 2900, loss : 0.00021
Epoch : 3000, loss : 0.00018
Epoch : 3100, loss : 0.00015
Epoch : 3200, loss : 0.00013
Epoch : 3300, loss : 0.00011
Epoch : 3400, loss : 0.000

In [41]:
df_X_ss = standard.transform(df)
df_y_mm = minmax.transform(y)
df_X_ss = Variable(torch.Tensor(df_X_ss))
df_y_mm = Variable(torch.Tensor(df_y_mm))
df_X_ss = torch.reshape(df_X_ss , (df_X_ss.shape[0],1,df_X_ss.shape[1]))

In [42]:
train_predict = model(df_X_ss)
data_predict = train_predict.data.detach().cpu().numpy()
dataY_plot = df_y_mm.data.numpy()

data_predict = minmax.inverse_transform(data_predict)

In [43]:
f1 = folium.PolyLine(locations = data_predict,color = 'green',tooltip = 'Predicted Trajectory')
f1.add_to(m)
m